# Pre-Processing

Goals of this notebook
: - Filter events from R1 abstraction level to R4 
- Extract features necessary for classification from raw message attributes 


## Filter
**Goal**: Define a filter that filters all events from low R1 abstraction level to R4 abstraction level
**Evaluation**: Compare R1 and R4 data for discarded events

In [4]:
import pandas as pd
import re
pd.options.mode.chained_assignment = None
import ast

In [5]:
# test with IL data 
#df_r4 = pd.read_csv('../../data_v3/ptp_il_tagged.csv')
#df_r1 = pd.read_csv('../../data/PTP-INTERLEAVED/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})

In [6]:
df_r4 = pd.read_csv('../../data_v3/ptp_extended_features.csv')
df_r1 = pd.read_csv('../../data/VALID/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})

### Pre-Processing

In [7]:
df_r1["pgsql.query"] = df_r1["MessageAttributes"].apply(lambda x: x.get("pgsql.query")).str.strip('\\xa').str.strip()


### Prerequisites
#### check that every frame of R4 is available in R1 

In [8]:
res = df_r4["frame.number"].isin(df_r1["frame.number"]).all()
print(f"Every value from df_r1 is present in df_r4: {res}")

Every value from df_r1 is present in df_r4: True


#### keep_event as evaluation criterion

In [9]:
df_r1["keep_event"] = df_r1["frame.number"].isin(df_r4["frame.number"])

In [10]:
df_r1["pgsql.query"] = df_r1["MessageAttributes"].apply(lambda x: x.get("pgsql.query")).str.strip('\\xa').str.strip()

### Filter by MessageType

In [11]:
df_r1[["MessageType", "keep_event"]].value_counts()

MessageType                                     keep_event
PgsqlResponse:Data row                          True          1026689
                                                False          484088
PgsqlResponse:Ready for query                   True           391192
PgsqlRequest:Simple query                       True           389504
PgsqlResponse:Command completion                True           387604
PgsqlResponse:Row description                   True           289975
PgsqlResponse:Parameter status                  True            82496
PgsqlResponse:Ready for query                   False           30080
PgsqlResponse:Command completion                False           29908
PgsqlRequest:Simple query                       False           28592
PgsqlResponse:Row description                   False           19861
PgsqlResponse:Authentication request            True             6587
Connection finish (FIN)                         True             6208
PgsqlResponse:Parameter status 

In [12]:
# Message Types to keep
message_type_filter = {
    "HttpRequest:POST /xmlrpc/2/common HTTP/1.1\\r\\n",
    "HttpRequest:POST /xmlrpc/2/object HTTP/1.1\\r\\n",
    "HttpResponse:HTTP/1.0 200 OK\\r\\n",
    "SmtpReassembledMessage",
    "PgsqlRequest:Simple query",
}

In [13]:
df_r1_filtered = df_r1[df_r1["MessageType"].isin(message_type_filter)]

In [14]:
df_r1_filtered[["keep_event", "MessageType"]].value_counts()

keep_event  MessageType                                   
True        PgsqlRequest:Simple query                         389504
False       PgsqlRequest:Simple query                          28592
True        HttpResponse:HTTP/1.0 200 OK\r\n                    2663
            HttpRequest:POST /xmlrpc/2/object HTTP/1.1\r\n      1842
            HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n       811
            SmtpReassembledMessage                               187
Name: count, dtype: int64

check if no keep events = True are dropped

In [15]:
df_r1.drop(df_r1_filtered.index)["keep_event"].value_counts()

keep_event
True     2224993
False     575391
Name: count, dtype: int64

the resulting elements in the original dataframe only have keep_events = False. 
**df_r1_filtered still contains all keep_events**

### Extend Filter by *PgsqlRequest:Simple query*

In [16]:
df_r1_psql = df_r1_filtered[df_r1_filtered["MessageType"] == "PgsqlRequest:Simple query"]

In [17]:
# keep first N 
df_r1_psql["pgsql.query_command_1"] = df_r1_psql["pgsql.query"].str.split().str[:1].str.join(" ")
df_r1_psql["pgsql.query_command_2"] = df_r1_psql["pgsql.query"].str.split().str[:2].str.join(" ")
df_r1_psql["pgsql.query_command_3"] = df_r1_psql["pgsql.query"].str.split().str[:3].str.join(" ")


def extract_pgsql_target(command):
    if command is not None:
        match = re.search(r'(?:INSERT INTO|UPDATE)\s+"?([^"\s]+)', command)
        return match.group(1) if match else None
    return None


df_r1_psql["pgsql.target"] = df_r1_psql["pgsql.query"].apply(extract_pgsql_target)

In [18]:
df_r1_psql[df_r1_psql["keep_event"]]

,Unnamed: 0,FileName,BusinessActivity,InstanceNumber,sniff_time,frame.number,synthetic_sniff_time,synthetic_sniff_time_str,session_generalized,HighestLayerProtocol,MessageType_WithRole,MessageType,MessageAttributes,pgsql.query,keep_event,pgsql.query_command_1,pgsql.query_command_2,pgsql.query_command_3,pgsql.target
5,5,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:39.600327,50,2020-12-21 21:40:11.290327,2020-12-21 21:40:11.290327,192.168.11.1-192.168.11.2 (1),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",BEGIN ISOLATION LEVEL REPEATABLE READ,True,BEGIN,BEGIN ISOLATION,BEGIN ISOLATION LEVEL,None
18,18,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:39.601493,56,2020-12-21 21:40:22.951493,2020-12-21 21:40:22.951493,192.168.11.1-192.168.11.2 (1),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",SELECT * FROM ir_cron\xa ...,True,SELECT,SELECT *,SELECT * FROM,None
22,22,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:39.602006,58,2020-12-21 21:40:28.082006,2020-12-21 21:40:28.082006,192.168.11.1-192.168.11.2 (1),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",COMMIT,True,COMMIT,COMMIT,COMMIT,None
48,48,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.677860,132,2020-12-22 11:46:31.697860,2020-12-22 11:46:31.697860,192.168.11.1-192.168.11.2 (2),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",BEGIN ISOLATION LEVEL REPEATABLE READ,True,BEGIN,BEGIN ISOLATION,BEGIN ISOLATION LEVEL,None
51,51,CreatePurchaseRequest_1.pcap,CreatePurchaseRequest,1,2020-12-21 08:08:44.678209,134,2020-12-22 11:46:35.188209,2020-12-22 11:46:35.188209,192.168.11.1-192.168.11.2 (2),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",select datname from pg_database where datdba=(...,True,select,select datname,select datname from,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3223961,3223961,SubmitPayment_9.pcap,SubmitPayment,9,2020-12-21 11:46:49.699675,4980,2021-01-10 13:46:27.869675,2021-01-10 13:46:27.869675,192.168.11.1-192.168.11.2 (7),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",BEGIN ISOLATION LEVEL REPEATABLE READ,True,BEGIN,BEGIN ISOLATION,BEGIN ISOLATION LEVEL,None
3223964,3223964,SubmitPayment_9.pcap,SubmitPayment,9,2020-12-21 11:46:49.699855,4982,2021-01-10 13:46:29.669855,2021-01-10 13:46:29.669855,192.168.11.1-192.168.11.2 (7),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...","SELECT ""res_users"".""id"" as ""id"",""res_users"".""p...",True,SELECT,"SELECT ""res_users"".""id""","SELECT ""res_users"".""id"" as",None
3223969,3223969,SubmitPayment_9.pcap,SubmitPayment,9,2020-12-21 11:46:49.701734,4985,2021-01-10 13:46:48.461734,2021-01-10 13:46:48.461734,192.168.11.1-192.168.11.2 (7),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...","SELECT ""res_partner"".""id"" as ""id"",""res_partner...",True,SELECT,"SELECT ""res_partner"".""id""","SELECT ""res_partner"".""id"" as",None
3223974,3223974,SubmitPayment_9.pcap,SubmitPayment,9,2020-12-21 11:46:49.704165,4990,2021-01-10 13:47:12.774165,2021-01-10 13:47:12.774165,192.168.11.1-192.168.11.2 (3),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",UPDATE ir_cron SET nextcall='2020-12-21 09:37:...,True,UPDATE,UPDATE ir_cron,UPDATE ir_

In [19]:
df_r1_psql[["pgsql.query_command_1", "keep_event"]].value_counts(dropna=False)

pgsql.query_command_1  keep_event
SELECT                 True          285881
BEGIN                  True           24621
DISCARD                True           23106
SELECT                 False          20138
UPDATE                 True           17539
COMMIT                 True           13769
ROLLBACK               True           11235
INSERT                 True            8196
select                 True            3567
BEGIN                  False           2487
DISCARD                False           2337
COMMIT                 False           1753
ROLLBACK               False            672
UPDATE                 False            599
WITH                   True             461
DELETE                 True             432
SAVEPOINT              True             431
DELETE                 False            337
RELEASE                True             150
select                 False            146
SELECT\xa              True              67
INSERT                 False             5

UPDATE and INSERT Commands are only commands that are present in both keep_event classes. However, heavily skew towards keeping them. 
-> Try including them, else wise rollback and try better filter

In [20]:
filtered_df = df_r1_psql[
    df_r1_psql['pgsql.query'].apply(lambda x: str(x).startswith(('UPDATE', 'INSERT')) if x is not None else True)]

In [21]:
# Extra Step for Email PTP Filtering
filtered_df = filtered_df[filtered_df["pgsql.target"] != "mail_tracking_value"]

In [22]:
filtered_df["keep_event"].value_counts()

keep_event
True     23736
False      599
Name: count, dtype: int64

filtered successful. Included the 15 UPDATE and INSERT Commands that were discard events

### Complete Filter Steps

In [23]:
# test with IL data 

df_r4 = pd.read_csv('../../data_v3/ptp_il_tagged.csv')
df_r1 = pd.read_csv('../../data/PTP-INTERLEAVED/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})

In [24]:
# mark keep events for evaluation
df_r1["keep_event"] = df_r1["frame.number"].isin(df_r4["frame.number"])

In [25]:
#1 
df_r1["pgsql.query"] = df_r1["MessageAttributes"].apply(lambda x: x.get("pgsql.query")).str.strip(' \\xa')
df_r1["pgsql.target"] = df_r1["pgsql.query"].apply(extract_pgsql_target)


#2 
# Message Types to keep
message_type_filter = {
    "HttpRequest:POST /xmlrpc/2/common HTTP/1.1\\r\\n",
    "HttpRequest:POST /xmlrpc/2/object HTTP/1.1\\r\\n",
    "HttpResponse:HTTP/1.0 200 OK\\r\\n",
    "SmtpReassembledMessage",
    "PgsqlRequest:Simple query",
}
df_r1_filtered = df_r1[df_r1["MessageType"].isin(message_type_filter)]

#3 
filtered_df = df_r1_filtered[
    df_r1_filtered['pgsql.query'].apply(lambda x: str(x).startswith(('UPDATE', 'INSERT')) if x is not None else True)]


# Extra Step for Email PTP Filtering
filtered_df = filtered_df[filtered_df["pgsql.target"] != "mail_tracking_value"]


filtered_df = filtered_df

In [26]:
filtered_df

,Unnamed: 0.1,Unnamed: 0,FileName,BusinessActivity,InstanceNumber,sniff_time,frame.number,synthetic_sniff_time,synthetic_sniff_time_str,session_generalized,HighestLayerProtocol,MessageType_WithRole,MessageType,MessageAttributes,keep_event,pgsql.query,pgsql.target
27,27,29,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.687676,96,2022-08-04 21:07:16.297676,2022-08-04 21:07:16.297676,192.168.11.2-192.168.11.10 (1),http,End Point (Employee)->Odoo Application:[HttpRe...,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,"{'': 'POST /xmlrpc/2/common HTTP/1.1\r\n', '_w...",True,None,None
82,82,90,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.958633,137,2022-08-04 21:52:26.138633,2022-08-04 21:52:26.138633,192.168.11.2-192.168.11.10 (1),http,Odoo Application->End Point (Employee):[HttpRe...,HttpResponse:HTTP/1.0 200 OK\r\n,"{'': 'HTTP/1.0 200 OK\r\n', '_ws.expert': 'Exp...",True,None,None
86,86,97,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.960536,145,2022-08-04 21:52:45.170536,2022-08-04 21:52:45.170536,192.168.11.2-192.168.11.10 (2),http,End Point (Employee)->Odoo Application:[HttpRe...,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,"{'': 'POST /xmlrpc/2/common HTTP/1.1\r\n', '_w...",True,None,None
188,188,210,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:22.298610,209,2022-08-04 22:49:06.248610,2022-08-04 22:49:06.248610,192.168.11.1-192.168.11.2 (7),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",True,"INSERT INTO ""res_users_log"" (""id"", ""create_uid...",res_users_log
258,258,282,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:22.413521,232,2022-08-04 23:08:15.473521,2022-08-04 23:08:15.473521,192.168.11.2-192.168.11.10 (2),http,Odoo Application->End Point (Employee):[HttpRe...,HttpResponse:HTTP/1.0 200 OK\r\n,"{'': 'HTTP/1.0 200 OK\r\n', '_ws.expert': 'Exp...",True,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224670,224670,234538,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:38:49.467434,112133,2022-08-21 23:39:41.657434,2022-08-21 23:39:41.657434,192.168.11.1-192.168.11.2 (4),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",True,"INSERT INTO ""account_payment"" (""id"", ""create_u...",account_payment
224675,224675,234543,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:38:49.471018,112135,2022-08-21 23:40:17.501018,2022-08-21 23:40:17.501018,192.168.11.1-192.168.11.2 (4),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",True,"INSERT INTO ""mail_followers"" (""id"", ""partner_i...",mail_followers
224680,224680,234548,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:38:49.471760,112137,2022-08-21 23:40:24.921760,2022-08-21 23:40:24.921760,192.168.11.1-192.168.11.2 (4),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",True,INSERT INTO mail_followers_mail_message_subtyp...,mail_followers_mail_message_subtype_rel
224763,224763,234635,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:38:49.487802,112175,2022-08-21 23:43:05.357802,2022-08-21 23:43:05.357802,192.168.11.1-192.168.11.2 (4),pgsql,Odoo Application->db Server/Mail Server:[Pgsql...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",True,"INSERT INTO ""mail_message"" (""id"", ""create

In [27]:
dropped_values = df_r1.drop(filtered_df.index)
dropped_values["keep_event"].value_counts()

keep_event
False    221171
True        118
Name: count, dtype: int64

dropped values only contain False keep events. 
No keep events are kept in the by the filter

In [28]:
filtered_df["keep_event"].value_counts()

keep_event
True    3599
Name: count, dtype: int64

all keep_events are included in the filtered Dataframe
The 15 Update and Insert Events are also included. 

## Feature Engineering
**Goal**: Extract Features required for classification from raw message attributes that are available in R1 abstraction level
**Evaluation**: Compare with R4 extracted features


### Prerequisites
The following features are required for classification:

event with roles
: acb

request method call
: def

selective_filter_data
: cde

origin_selective_filter_data
: def


In [29]:
# test with IL data 

df_r4 = pd.read_csv('../../data_v3/ptp_il_tagged.csv')
df = filtered_df

### Feature Extraction

#### Event with Roles
Captures the involved roles as well as the event that takes place

In [30]:
# introduce isolated Role Column
df["roles"] = df["MessageType_WithRole"].str.split(":").str[0]

In [31]:



def extract_pgsql_target(command):
    """
    Extracts the target of the pgsql command. 
    :param command: pgsql query
    :return: the target of the pgsql command. 
    """
    if command is not None:
        match = re.search(r'(?:INSERT INTO|UPDATE)\s+"?([^"\s]+)', command)
        return match.group(1) if match else None
    return None


# Extract first Keyword from Query
df["pgsql.query_0"] = df["pgsql.query"].str.split(" ").str[0]

# Extract target from Query
df["pgsql.target"] = df['pgsql.query'].apply(extract_pgsql_target)

In [32]:
def set_event_with_roles(row):
    sep = ": "
    res = [row["MessageType_WithRole"]]
    if row["pgsql.query_0"]:
        res.append(row["pgsql.query_0"])
    if row["pgsql.target"]:
        res.append(row["pgsql.target"])
    return ": ".join(res)




df["event_with_roles"] = df.apply(set_event_with_roles, axis=1)

#### HTTP Features
extract additional features for HTTP requests

In [33]:
# Extract XML http file data from Message Attributes
df["http_file_data"] = df["MessageAttributes"].apply(lambda x: x.get("http.file_data", None))
# Only for response packets: Specifies the request packet a response is referring to
df["http_request_in"] = df["MessageAttributes"].apply(lambda x: x.get("http.request_in", pd.NA)).astype("Int64")

In [34]:
from lxml import etree

#TODO Adjust
setting = "PTP"


def extract_http_file_data(data):
    

    if data is not None:
        #print(content)
        file_data = []
        selective_file_data = None
        method_call = None

        content = data.replace("\\xa", "")
        root = etree.fromstring(content)

        method_type = root.find(".").tag

        # extract method call
        if method_type == "methodCall":
            method_name = root.find(".//methodName")
            if method_name is not None:
                method_call = method_name.text

        elif method_type == "methodResponse":
            method_call = "response"

            # Extract parameters
        params = root.find(".//params")
        if params is not None:
            file_data = root.find(".//params").xpath('.//text()')
            # process data 
            if method_call == "execute_kw" or method_call == "id":
                if 'salary_proposed' in file_data:
                    selective_file_data = "_".join([*file_data[3:5], file_data[-2]])
                elif 'search_read' in file_data:
                    if setting == "HR":
                        selective_file_data = "_".join([*file_data[3:5], file_data[-1]])
                    else: 
                        selective_file_data = "_".join([*file_data[3:5]])
                elif 'salary_expected' in file_data:
                    selective_file_data = "_".join([*file_data[3:5], file_data[6]])
                elif 'mail.activity' in file_data:
                    selective_file_data = "_".join(file_data[3:5]) if 'action_done' in file_data else "_".join(
                        [*file_data[3:5], file_data[-1]])
                elif 'stage_id' in file_data:
                    selective_file_data = "_".join([*file_data[3:5], *file_data[-2:]])
                else:
                    selective_file_data = "_".join(file_data[3:5])
            elif method_call == "response":
                method_call = file_data[0]
                if method_call == "id":
                    if 'name' in file_data: 
                        pass
                    if 'salary_expected' in file_data: 
                        selective_file_data = "salary_expected"  
                        
                    else: 
                        selective_file_data = file_data[2]
                    # if 'picking_ids' in file_data: 
                    #     selective_file_data = "picking_ids"
                    # if 'move_line_ids' in file_data: 
                    #     selective_file_data = "move_line_ids"
                    # if ''
                elif method_call.isdigit():
                    method_call = "IsNumber"
                    selective_file_data = "IsNumber"
                elif method_call == "server_version":
                    selective_file_data = "server_version"
                else:
                    selective_file_data = "_".join(file_data[-2:])


            elif method_call == "version":
                selective_file_data = "version"

        elif method_call == "response":
            method_call = "faultCode"
            selective_file_data = "faultCode"

        return method_call, file_data, selective_file_data

    return "","",""


#df[["request_method_call", "file_data", "selective_file_data"]] = df.apply(lambda x: extract_http_file_data(x["http_file_data"]), axis=1,result_type="expand")

In [35]:
df_r4[df_r4["request_method_call"].fillna("").str.isnumeric()]

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,BusinessActivity,InstanceNumber,sniff_time,frame.number,synthetic_sniff_time,event_with_roles,request_method_call,...,account_move_id,stock_move_id,group_id,picking_id,invoice_id,stock_picking_id,account_invoice_id,stock_move_line_id,date,real_activity_action
4,4,4,4,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:22.413521,232,2022-08-04 23:08:15.473521,Odoo Application->End Point (Employee): [HttpR...,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoAction
19,19,19,19,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:23.562064,778,2022-08-05 02:19:42.052064,Odoo Application->End Point (Employee): [HttpR...,375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoAction
41,41,41,41,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:24.112864,1322,2022-08-05 03:51:30.602864,Odoo Application->End Point (Employee): [HttpR...,399,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Activity End
46,46,46,46,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:26.451943,1462,2022-08-05 10:21:23.731943,Odoo Application->End Point (Procurement): [Ht...,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoAction
53,53,53,53,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:26.669104,1658,2022-08-05 10:57:35.559104,Odoo Application->End Point (Procurement): [Ht...,338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoAction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3632,3632,3632,3632,order_to_cash_interleaved_10cases,2022,2022-08-03 13:38:48.099459,109851,2022-08-21 19:51:40.539459,Odoo Application->End Point (Finance): [HttpRe...,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoAction
3651,3651,3651,3651,order_to_cash_interleaved_10cases,2022,2022-08-03 13:38:48.578463,110583,2022-08-21 21:11:31.058463,Odoo Application->End Point (Finance): [HttpRe...,253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoAction
3664,3664,3664,3664,order_to_cash_interleaved_10cases,2022,2022-08-03 13:38:48.816709,110961,2022-08-21 21:51:13.756709,Odoo Application->End Point (Finance): [HttpRe...,257,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoAction
3710,3710,3710,3710,order_to_cash_interleaved_10cases,2022,2022-08-03 13:38:49.416259,112076,2022-08-21 23:31:09.856259,Odoo Application->End Point (Finance): [HttpRe...,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoAction


In [36]:
df_r4[["event_with_roles","request_method_call","file_data"]].value_counts(dropna=False)

event_with_roles                                                                                     request_method_call  file_data                                                                                               
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['purchase_order']]       NaN                  []                                                                                                          400
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:INSERT:['mail_message']]         NaN                  []                                                                                                          315
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['purchase_order_line']]  NaN                  []                                                                                                          290
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPD

In [37]:
# # repalce request method call number 
# def replace_numeric(value):
#     return '' if pd.to_numeric(value, errors='coerce') == value else value
# 
# 
# df["request_method_call"][pd.notnull(pd.to_numeric(df["request_method_call"], errors='coerce'))] = ""

In [38]:
def merge_origin_request(row, df):
    if pd.notna(row["http_request_in"]):
        instance_number = row["InstanceNumber"]
        request_frame = row["http_request_in"]
        business_activity = row["BusinessActivity"]
        try:
            row = df[(df["InstanceNumber"] == instance_number) & (df["frame.number"] == request_frame) & (
                    df["BusinessActivity"] == business_activity)].iloc[0]
            selective_file_data = row["selective_file_data"]
            method_call = row["request_method_call"]
            return method_call, selective_file_data
        except IndexError:
            pass
    return None, None


#df[["origin_method", "origin_file_data"]] = df.apply(merge_origin_request, axis=1, df=df, result_type="expand").fillna("")

In [39]:
df

,Unnamed: 0.1,Unnamed: 0,FileName,BusinessActivity,InstanceNumber,sniff_time,frame.number,synthetic_sniff_time,synthetic_sniff_time_str,session_generalized,...,MessageType,MessageAttributes,keep_event,pgsql.query,pgsql.target,roles,pgsql.query_0,event_with_roles,http_file_data,http_request_in
27,27,29,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.687676,96,2022-08-04 21:07:16.297676,2022-08-04 21:07:16.297676,192.168.11.2-192.168.11.10 (1),...,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,"{'': 'POST /xmlrpc/2/common HTTP/1.1\r\n', '_w...",True,None,None,End Point (Employee)->Odoo Application,None,End Point (Employee)->Odoo Application:[HttpRe...,<?xml version='1.0'?>\xa<methodCall>\xa<method...,<NA>
82,82,90,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.958633,137,2022-08-04 21:52:26.138633,2022-08-04 21:52:26.138633,192.168.11.2-192.168.11.10 (1),...,HttpResponse:HTTP/1.0 200 OK\r\n,"{'': 'HTTP/1.0 200 OK\r\n', '_ws.expert': 'Exp...",True,None,None,Odoo Application->End Point (Employee),None,Odoo Application->End Point (Employee):[HttpRe...,<?xml version='1.0'?>\xa<methodResponse>\xa<pa...,96
86,86,97,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:21.960536,145,2022-08-04 21:52:45.170536,2022-08-04 21:52:45.170536,192.168.11.2-192.168.11.10 (2),...,HttpRequest:POST /xmlrpc/2/common HTTP/1.1\r\n,"{'': 'POST /xmlrpc/2/common HTTP/1.1\r\n', '_w...",True,None,None,End Point (Employee)->Odoo Application,None,End Point (Employee)->Odoo Application:[HttpRe...,<?xml version='1.0'?>\xa<methodCall>\xa<method...,<NA>
188,188,210,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:22.298610,209,2022-08-04 22:49:06.248610,2022-08-04 22:49:06.248610,192.168.11.1-192.168.11.2 (7),...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",True,"INSERT INTO ""res_users_log"" (""id"", ""create_uid...",res_users_log,Odoo Application->db Server/Mail Server,INSERT,Odoo Application->db Server/Mail Server:[Pgsql...,None,<NA>
258,258,282,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:36:22.413521,232,2022-08-04 23:08:15.473521,2022-08-04 23:08:15.473521,192.168.11.2-192.168.11.10 (2),...,HttpResponse:HTTP/1.0 200 OK\r\n,"{'': 'HTTP/1.0 200 OK\r\n', '_ws.expert': 'Exp...",True,None,None,Odoo Application->End Point (Employee),None,Odoo Application->End Point (Employee):[HttpRe...,<?xml version='1.0'?>\xa<methodResponse>\xa<pa...,145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224670,224670,234538,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:38:49.467434,112133,2022-08-21 23:39:41.657434,2022-08-21 23:39:41.657434,192.168.11.1-192.168.11.2 (4),...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",True,"INSERT INTO ""account_payment"" (""id"", ""create_u...",account_payment,Odoo Application->db Server/Mail Server,INSERT,Odoo Application->db Server/Mail Server:[Pgsql...,None,<NA>
224675,224675,234543,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:38:49.471018,112135,2022-08-21 23:40:17.501018,2022-08-21 23:40:17.501018,192.168.11.1-192.168.11.2 (4),...,PgsqlRequest:Simple query,"{'pgsql.type': 'Simple query', 'pgsql.length':...",True,"INSERT INTO ""mail_followers"" (""id"", ""partner_i...",mail_followers,Odoo Application->db Server/Mail Server,INSERT,Odoo Application->db Server/Mail Server:[Pgsql...,None,<NA>
224680,224680,234548,async_2022_08_03-0336_10_cases_10_interval.pcap,order_to_cash_interleaved_10cases,2022,2022-08-03 13:38:49.471760,112137,2022-08-21 23:40:24.921760,2022-08-21 23:40:24.921760,192.168.11.1-192.168.11.2 (4),...,PgsqlRequest:Simple query,"{'pgsql.ty

## Complete Pipeline

In [40]:
# test with IL data 

#df_r4 = pd.read_csv('../../data_v3/hr_il_tagged.csv')
df_r1_HR = pd.read_csv('../../data/HR-INTERLEAVED/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})

In [41]:
# test with IL data 
df_train_in = pd.read_csv('../../data/VALID/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})
df_r4 = pd.read_csv('../../data_v3/ptp_il_tagged.csv')
df_r1 = pd.read_csv('../../data/PTP-INTERLEAVED/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})

In [42]:
# mark keep events for evaluation
df_r1["keep_event"] = df_r1["frame.number"].isin(df_r4["frame.number"])

In [43]:
import re


def apply_filter(df):
    #1 preprocess
    df["pgsql.query"] = df["MessageAttributes"].apply(lambda x: x.get("pgsql.query")).str.strip(' \\xa')
    
    df["pgsql.target"] = df["pgsql.query"].apply(extract_pgsql_target)

    #2 filter by message type
    # Message Types to keep
    message_type_filter = {
        "HttpRequest:POST /xmlrpc/2/common HTTP/1.1\\r\\n",
        "HttpRequest:POST /xmlrpc/2/object HTTP/1.1\\r\\n",
        "HttpResponse:HTTP/1.0 200 OK\\r\\n",
        "SmtpReassembledMessage",
        "PgsqlRequest:Simple query",
    }
    df = df[df["MessageType"].isin(message_type_filter)]

    #3 keep PGSQL query that are either UPDATE or INSERT
    df = df[df['pgsql.query'].apply(lambda x: str(x).startswith(('UPDATE', 'INSERT')) if x is not None else True)]
    
    # Extra Step for Email PTP Filtering
    #df = df[~df["pgsql.target"].isin(["mail_tracking_value", "fetchmail_server", "ir_cron", "ir_config_parameter"]) ]
    df = df[~df["pgsql.target"].isin(["fetchmail_server", "ir_cron", "ir_config_parameter"]) ]

    return df


def extract_features(df):
    df["pgsql.query"] = df["MessageAttributes"].apply(lambda x: x.get("pgsql.query")).str.strip(' \\xa')

    # event with roles feature 
    df["roles"] = df["MessageType_WithRole"].str.split(":").str[0]

    # Extract first Keyword from Query
    df["pgsql.query_0"] = df["pgsql.query"].str.split(" ").str[0]

    # Extract target from Query
    df["pgsql.target"] = df['pgsql.query'].apply(extract_pgsql_target)

    # Build Event with roles feature
    df["event_with_roles"] = df.apply(set_event_with_roles, axis=1)

    # Extract XML http file data from Message Attributes
    df["http_file_data"] = df["MessageAttributes"].apply(lambda x: x.get("http.file_data", None))
    # Only for response packets: Specifies the request packet a response is referring to
    df["http_request_in"] = df["MessageAttributes"].apply(lambda x: x.get("http.request_in", pd.NA)).astype("Int64")

    # Extract HTTP Features
    df[["request_method_call", "file_data", "selective_file_data"]] = df.apply(lambda x: extract_http_file_data(x["http_file_data"]), axis=1,
                                                                               result_type="expand")

    #df["request_method_call"][pd.notnull(pd.to_numeric(df["request_method_call"], errors='coerce'))] = ""

    # Map Origin Requests
    df[["origin_method", "origin_file_data"]] = df.apply(merge_origin_request, axis=1, df=df,
                                                         result_type="expand").fillna("")

    # only keep necessary fields 

    cols = ["BusinessActivity",
            "InstanceNumber",
            "frame.number",
            "synthetic_sniff_time",
            "event_with_roles",
            "request_method_call",
            "file_data",
            "selective_file_data",
            "origin_method",
            "origin_file_data"]

    return df[cols].fillna("")


def pre_process(df):
    df = apply_filter(df)
    df = extract_features(df)

    return df


In [44]:
df_r1_pp = pre_process(df_r1)

In [45]:
df_r1_pp["selective_file_data"].value_counts()

selective_file_data
                                       3170
IsNumber                                262
server_version                           64
version                                  63
purchase.requisition_write               38
name                                     35
purchase.order.line_create               29
purchase.order_create                    27
product.product_search_read              20
purchase.order_search_read               16
res.partner_search_read                  10
sale.order_write                         10
default_code                             10
sale.order.line_create                   10
sale.order_create                        10
purchase.requisition.line_create         10
purchase.requisition_create               9
faultCode                                 8
account.payment_create                    8
account.invoice_action_invoice_open       8
account.invoice.line_create               8
account.invoice_create                    8
account.jour

In [46]:
df_train_pp = pre_process(df_train_in)

In [47]:
df_r1_pp["selective_file_data"].isin(df_train_pp["selective_file_data"]).all()

True

In [48]:
df_r1_pp["event_with_roles"].isin(df_train_pp["event_with_roles"]).all()

True

# Analyze PTP Activities for looping fine granular events

In [53]:
# TODO Show loops in fine granular events that make it hard to analyze PTP Data

In [52]:
df_r1_pp[["event_with_roles","request_method_call","selective_file_data","origin_method","origin_file_data"]]

,event_with_roles,request_method_call,selective_file_data,origin_method,origin_file_data
27,End Point (Employee)->Odoo Application:[HttpRe...,version,version,,
82,Odoo Application->End Point (Employee):[HttpRe...,server_version,server_version,version,version
86,End Point (Employee)->Odoo Application:[HttpRe...,authenticate,,,
188,Odoo Application->db Server/Mail Server:[Pgsql...,,,,
258,Odoo Application->End Point (Employee):[HttpRe...,IsNumber,IsNumber,authenticate,
...,...,...,...,...,...
224670,Odoo Application->db Server/Mail Server:[Pgsql...,,,,
224675,Odoo Application->db Server/Mail Server:[Pgsql...,,,,
224680,Odoo Application->db Server/Mail Server:[Pgsql...,,,,
224763,Odoo Application->db Server/Mail Server:[Pgsql...,,,,
